<a href="https://colab.research.google.com/github/VickyAlves/Power-BI-Tests/blob/main/Desafio_DIO_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
import zipfile
import pathlib

# -----------------------
# 1. EXTRAIR O ZIP
# -----------------------
zip_path = "/content/cats and dogs/kagglecatsanddogs_5340.zip"
extract_dir = "/content/cats_and_dogs_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Caminho para a pasta que contém "Cat" e "Dog"
data_dir = os.path.join(extract_dir, "PetImages")
data_dir = pathlib.Path(data_dir)

# -----------------------
# 2. REMOVER ARQUIVOS CORROMPIDOS
# -----------------------
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = data_dir / folder_name
    for fname in os.listdir(folder_path):
        fpath = folder_path / fname
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
            fobj.close()
        except:
            is_jfif = False
        if not is_jfif:
            num_skipped += 1
            os.remove(fpath)
print(f"Removidos {num_skipped} arquivos corrompidos.")

# -----------------------
# 3. PARÂMETROS
# -----------------------
img_size = (224, 224)
batch_size = 32
val_split = 0.2
seed = 42

# -----------------------
# 4. CARREGAR DADOS
# -----------------------
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# -----------------------
# 5. TRANSFER LEARNING COM MOBILENETV2
# -----------------------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

global_avg_layer = layers.GlobalAveragePooling2D()
prediction_layer = layers.Dense(1, activation='sigmoid')

model = models.Sequential([
    base_model,
    global_avg_layer,
    layers.Dropout(0.2),
    prediction_layer
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# -----------------------
# 6. TREINAMENTO
# -----------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

# -----------------------
# 7. FINE-TUNING (opcional)
# -----------------------
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1
)

# -----------------------
# 8. SALVAR MODELO
# -----------------------
model.save("cat_dog_transfer_learning.h5")


BadZipFile: File is not a zip file

In [ ]:
history = model.fit(...)


In [ ]:
history_finetune = model.fit(...)


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Caminho para imagem de teste
img_path = "/content/imagem_2025-08-11_211926681.png"  # troque para seu arquivo

# Pré-processar a imagem
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Prever
pred = model.predict(img_array)

if pred[0][0] > 0.5:
    print(f"É um DOG ({pred[0][0]:.2f})")
else:
    print(f"É um CAT ({1 - pred[0][0]:.2f})")
